In [3]:
%load_ext autoreload
%autoreload 2

import pennylane as qml
import math
from pennylane import qaoa
from pennylane import numpy as np
import matplotlib.pyplot as plt
from vqa.hamiltonian.protein_folding import protein_folding_hamiltonian
from vqa.hamiltonian.Protein_models.CoordinateBased_HPLattice import CoordinateBased_HPLattice
from vqa.utils.protein_utils import *
from collections import Counter

import GPyOpt
    
from GPyOpt.methods import BayesianOptimization

In [4]:
# Make an instance for the protein
sequence = [1,0,0,1] #HPPH
L1 = 2
L2 = 2

lambda_vector = (2.1, 2.4, 3)

protein = CoordinateBased_HPLattice((L1, L2), sequence = sequence, lambda_vector = lambda_vector)
print(protein)
protein.calc_solution_sets()
feasible_set = np.array(protein.feasible_set)


O:
[-2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1]
T:
[[ 0.   4.2  0.   0.   2.4  0.  -1.  -1. ]
 [ 4.2  0.   0.   0.   0.   2.4 -1.  -1. ]
 [ 0.   0.   0.   4.2  0.   0.   2.4  0. ]
 [ 0.   0.   4.2  0.   0.   0.   0.   2.4]
 [ 2.4  0.   0.   0.   0.   4.2  0.   0. ]
 [ 0.   2.4  0.   0.   4.2  0.   0.   0. ]
 [-1.  -1.   2.4  0.   0.   0.   0.   4.2]
 [-1.  -1.   0.   2.4  0.   0.   4.2  0. ]]
Dn:
[2, 2, 2, 2]


In [5]:
# Make into Hamiltonian
H_cost = protein_folding_hamiltonian(protein)
num_qubits = protein.num_bits
qubits = range(protein.num_bits)

H_mixer = qaoa.x_mixer(qubits)

In [6]:
dev = qml.device('default.qubit', wires = qubits)
#dev = qml.device('default.qubit', wires = qubits, shots = 1000) #stochastic, use this with number of samples??

# Put the Mixer and Cost Hamiltonians in a layer so that we can repeat it
def qaoa_layer(gamma, beta):
    qaoa.cost_layer(gamma, H_cost)
    qaoa.mixer_layer(beta, H_mixer)
    
# Then repeat it in a circuit with starting in a super position of all bitstrings
def circuit(params):     # Gamma and Beta values can be put together to be an array of parameters
    for q in qubits:     # To start in a superposition we place a Hadamard on all qubits
        qml.Hadamard(wires = q)
    qml.layer(qaoa_layer, len(params[0]), params[0], params[1])
    
# Obtains the probability for all the states
@qml.qnode(dev)
def get_probs(params):
    circuit(params)
    return qml.probs(wires = qubits)

# CVaR function for known energies and probabilities
def CVaR(params, alpha=0.25):
    #circuit(params)
    
    Energy_list = get_energies_index_states(H_cost)
    Energy_array = np.column_stack((Energy_list, get_probs(params)))
    Energy_array = Energy_array[Energy_array[:, 0].argsort()]
    
    k = 0
    p_sum = 0
    while p_sum < alpha and k<len(Energy_array[:,0]):
        p_sum = p_sum + Energy_array[k,1]
        k = k + 1
    
    Energy_vector = Energy_array[0:k-1, 0]
    Probability_vector = Energy_array[0:k-1, 1]
    
    exp_CVaR = np.dot(Energy_vector, Probability_vector)/p_sum
    
    #print(params)
    print(type(exp_CVaR))    
    return exp_CVaR

#@qml.qnode(dev)
def CVaRsamp(params, alpha=0.25, n=1000):#n=antal samples
    
    #circuit(params)
    
    probs = get_probs(params)  # hämta sannolikhetsfördelningen (matris, första värdet är p(00000000), andra är p(00000001) osv)
    
    index_samples= np.random.choice(np.arange(len(probs), dtype=int), size=n, replace=True,p=probs)  # tar n samples av probs, ger skum lista med index motsvarande konfiguration (index 0 är tillståndet 00000000, index 1 =00000001 osv)
    energy_of_samples=energies_of_set(protein.get_solution_set(), H_cost,8) [index_samples]  #ger en lista där index i index_samples är utbytta mot deras motsvarande energi. 
    sorted_energy_samples=sort_over_threshhold(energy_of_samples,-10)[0] #sorterar hela energilistan
    K=int(alpha*n) #antal samples att ta väntevärde över. 
    summa=sum(sorted_energy_samples[:K]) #summera de K minsta energierna.
    expvalue=summa/K
    print(type(expvalue))
    return expvalue

initial_params = np.array([[0.5, 0.5], [0.5, 0.5]], requires_grad = True)
print(CVaR(initial_params))
print(CVaRsamp(initial_params))

<class 'pennylane.numpy.tensor.tensor'>
-1.3334689968290745
<class 'pennylane.numpy.tensor.tensor'>
-1.5423999999999962


In [7]:
#Här kommer GPyOpt-skiten

def trainGPyOpt(p, stepsize = 0.01,
steps = 10,
huristic = 'average cost',
n = None,
swarm = 1,
initial_design_type = 'latin',
acquisition_type='EI',
exact_feval = False,
given_init_params = np.full((2, 1), None),
plot = False,
verbose = True,
gamma_min = 0,
gamma_max = np.pi*2,
beta_min = 0,
beta_max = np.pi*2
):
#'''
#swarm = 0 and given init params will not swarm. 
#'''

    def wrap_cost(params_list, p):
        cost_function = average_cost #change to CVaR(samp)
        gamma_list = params_list[0][:p]
        beta_list = params_list[0][p:]
        params = np.array([gamma_list, beta_list])
        return cost_function(params)

    gamma = []
    beta = []
    
    for i in range(self.p):
        gamma += [{'name': 'gamma'+str(i+1),
        'type': 'continuous', 'domain': (gamma_min, gamma_max)}]
        beta += [{'name': 'beta'+str(i+1),
        'type': 'continuous', 'domain': (beta_min, beta_max)}]
        bounds = gamma + beta
        myBopt = GPyOpt.methods.BayesianOptimization(wrap_cost,
        domain=bounds,
        initial_design_numdata = swarm,
        verbosity = verbose,
        initial_design_type = initial_design_type,
        acquisition_type = acquisition_type,
        exact_feval = exact_feval)
        if swarm == 0 and given_init_params.all() != None:
            print('Given init')
            myBopt.X = np.array([given_init_params.flatten().tolist()])
            myBopt.Y = wrap_cost(np.array([given_init_params.flatten().tolist()])).reshape(-1, 1)
    
        myBopt.run_optimization(max_iter = steps, verbosity = verbose)
        if p == 1:
            myBopt.plot_acquisition()
            plt.savefig('acq.pdf')
    
        if plot:
            myBopt.plot_convergence()
            plt.savefig('conv.pdf')
            plot_cost(cost_vector, new_fig = False, save = True, name = heuristic)
    
    
        best_params = np.array([myBopt.x_opt[:self.p], myBopt.x_opt[self.p:]])
        params_vector, cost_vector = myBopt.get_evaluations()
        t1 = time.time()
    
    print("="*20)
    print("Value of (gamma, beta) that minimises the objective:"+str(myBopt.x_opt))    
    print("Minimum value of the objective: "+str(myBopt.fx_opt)+' Success prob: '+ str(self.success_prob(best_params)))     
    print("="*20)
    
    return best_params, cost_vector, params_vector

In [9]:
p = 2
trainGPyOpt(p)


NameError: name 'self' is not defined